## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-21-22-50-58 +0000,nypost,JPMorgan admits it shuttered Trump’s accounts ...,https://nypost.com/2026/02/21/us-news/jpmorgan...
1,2026-02-21-22-44-17 +0000,startribune,You’ll never guess what’s making those weird b...,https://www.startribune.com/blue-snow-minnesot...
2,2026-02-21-22-31-54 +0000,nyt,Nine Bodies of Skiers Recovered in Sierra Neva...,https://www.nytimes.com/2026/02/21/us/lake-tah...
3,2026-02-21-22-19-39 +0000,nypost,Bill Maher rips into Billie Eilish and Chappel...,https://nypost.com/2026/02/21/us-news/bill-mah...
4,2026-02-21-22-17-31 +0000,nyt,Trump Says He Will Raise Global Tariff to 15 P...,https://www.nytimes.com/2026/02/21/business/tr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2486/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,48
194,court,26
193,supreme,21
198,tariffs,18
42,tariff,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
205,2026-02-21-00-24-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...,206
140,2026-02-21-10-30-00 +0000,wsj,The Supreme Court’s ruling that most of Trump’...,https://www.wsj.com/economy/trade/trump-lost-o...,157
73,2026-02-21-17-23-00 +0000,wsj,President Trump said he would increase to 15% ...,https://www.wsj.com/politics/policy/trump-incr...,153
82,2026-02-21-16-56-26 +0000,cbc,Trump says he's raising new global tariff rate...,https://www.cbc.ca/news/politics/trump-global-...,152
84,2026-02-21-16-46-07 +0000,wapo,Trump’s response to Supreme Court tariff rulin...,https://www.washingtonpost.com/business/2026/0...,142


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
205,206,2026-02-21-00-24-00 +0000,wsj,President Trump had just started a closed-door...,https://www.wsj.com/politics/policy/the-moment...
25,61,2026-02-21-20-50-06 +0000,bbc,Trump says he will increase his new global tar...,https://www.bbc.com/news/articles/cn8z48xwqn3o...
68,37,2026-02-21-17-36-48 +0000,nyt,"After Tariff Decision, What Happens to Trump’s...",https://www.nytimes.com/2026/02/20/business/ec...
15,35,2026-02-21-21-42-40 +0000,nypost,Ski guides killed in Lake Tahoe avalanche iden...,https://nypost.com/2026/02/21/us-news/blackbir...
7,32,2026-02-21-22-09-19 +0000,bbc,Iran students stage first large anti-governmen...,https://www.bbc.com/news/articles/c5yj2kzkrj0o...
23,26,2026-02-21-20-54-20 +0000,nypost,Israeli strikes on Hezbollah kill 8 as terror ...,https://nypost.com/2026/02/21/world-news/israe...
9,25,2026-02-21-22-02-59 +0000,startribune,Minnesota bars opening extra early Sunday for ...,https://www.startribune.com/minnesota-bars-ope...
29,24,2026-02-21-20-24-11 +0000,nypost,Ex-Prince Andrew hits rock bottom as Brits mov...,https://nypost.com/2026/02/21/world-news/brits...
142,24,2026-02-21-10-05-15 +0000,nypost,Justice Department swiftly fires lawyer chosen...,https://nypost.com/2026/02/21/us-news/virginia...
180,24,2026-02-21-02-51-32 +0000,nypost,Iranian forces hack out wombs of female protes...,https://nypost.com/2026/02/20/world-news/impri...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
